In [1]:
NEO4J_URI="neo4j+s://97eb342c.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="MA4GQtOPNwVtSJGZADQcQYMTZHqz8GokOUehBvcnCTc"

In [2]:
import os 
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [3]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [4]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [5]:
graph.query(moview_query)

Failed to write data to connection ResolvedIPv4Address(('34.126.161.242', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687)))
Failed to write data to connection IPv4Address(('97eb342c.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687)))


[]

In [6]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
Company {name: STRING}
Student {POB: STRING, name: STRING, YOB: INTEGER}
Country {name: STRING}
City {name: STRING}
Person {name: STRING, Born: INTEGER, born: INTEGER}
Movie {Released: INTEGER, Title: STRING, id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
USER {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}
Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
(:USER)-[:POSTED]->(:Post)
(:USER)-[:LIKES]->(:USER)
(:USER)-[:FRIEND]->(:USER)


In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [8]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025003D2FB90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025003D15510>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [9]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000025069692550>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025003D2FB90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025003D15510>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [10]:
response = chain.run("Who directed the movie Inception?")
print(response)


C:\Users\91798\AppData\Local\Temp\ipykernel_20156\1809615367.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run("Who directed the movie Inception?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie{title:"Inception"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[]

> Finished chain.
I don't know the answer. 



In [11]:
chain.run("Who was the director of the movie Casino")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


'Martin Scorsese  \n'

In [13]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) WHERE p.name IS NOT NULL RETURN count(DISTINCT p)AS artistCount
Full Context:
[{'artistCount': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}

In [14]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(a:Person) RETURN a.name 

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods \n'}